In [4]:
!pip install gradio pandas matplotlib openpyxl -q

import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime

DATA_FILE = "bmi_data.csv"
USERS_FILE = "users.csv"

COLUMNS = [
    "username",
    "age",
    "gender",
    "height_cm",
    "weight_kg",
    "bmi",
    "category",
    "datetime"
]

def load_data():
    if os.path.exists(DATA_FILE):
        try:
            df = pd.read_csv(DATA_FILE)
            for c in COLUMNS:
                if c not in df.columns:
                    df[c] = None
            return df[COLUMNS]
        except:
            return pd.DataFrame(columns=COLUMNS)
    return pd.DataFrame(columns=COLUMNS)

def save_data(df):
    df.to_csv(DATA_FILE, index=False)

def load_users():
    if os.path.exists(USERS_FILE):
        try:
            df = pd.read_csv(USERS_FILE)
            if "username" not in df.columns:
                return pd.DataFrame(columns=["username", "password"])
            return df
        except:
            return pd.DataFrame(columns=["username", "password"])
    return pd.DataFrame(columns=["username", "password"])

def save_users(df):
    df.to_csv(USERS_FILE, index=False)

# --------------------------
# LOGIN / REGISTER
# --------------------------
def login_or_register(username, password):
    username = (username or "").strip()
    if not username:
        return "Please enter a username.", "", ""
    if not password:
        return "Please enter a password.", "", ""

    df = load_users()
    if not df.empty and "username" in df.columns:
        mask = df["username"].astype(str).str.lower() == username.lower()
    else:
        mask = pd.Series([], dtype=bool)

    if mask.any():
        row = df[mask].iloc[0]
        if str(row["password"]) == str(password):
            return f"Logged in as {row['username']}", row["username"], row["username"]
        return "Incorrect password.", "", ""

    new_row = {"username": username, "password": password}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    save_users(df)
    return f"User '{username}' registered successfully.", username, username

def handle_login(username, password):
    msg, user_state, display = login_or_register(username, password)
    if user_state:
        tab_vis = gr.update(visible=True)
    else:
        tab_vis = gr.update(visible=False)
    return msg, user_state, display, tab_vis, tab_vis, tab_vis, user_state

# --------------------------
# BMI CALCULATIONS
# --------------------------
def calculate_bmi(weight_kg, height_cm):
    height_m = height_cm / 100
    return round(weight_kg / (height_m ** 2), 2)

def classify_bmi(bmi):
    if bmi < 18.5: return "Underweight"
    if bmi < 25: return "Normal weight"
    if bmi < 30: return "Overweight"
    return "Obese"

def advice_for_category(category):
    if category == "Underweight":
        return "Increase calorie intake with healthy foods."
    if category == "Normal weight":
        return "Great! Maintain your lifestyle."
    if category == "Overweight":
        return "Try regular exercise and healthy diet."
    return "Consult a healthcare provider for guidance."

def make_result_card(username, bmi, category):
    chip_class = {
        "Underweight": "bmi-chip-under",
        "Normal weight": "bmi-chip-normal",
        "Overweight": "bmi-chip-over",
        "Obese": "bmi-chip-obese"
    }.get(category, "bmi-chip-normal")

    name = username or "Friend"

    html = f"""
    <div class="bmi-card">
        <div class="bmi-header">
            <span class="bmi-hello">Hi, {name} 👋</span>
            <span class="bmi-small-text">Here is your BMI summary</span>
        </div>
        <div class="bmi-main">
            <div class="bmi-number">{bmi}</div>
            <div class="bmi-label-row">
                <span class="bmi-label">Your BMI</span>
                <span class="bmi-chip {chip_class}">{category}</span>
            </div>
        </div>
        <div class="bmi-footer-text">
            This is your current body mass index based on the details you entered.
        </div>
    </div>
    """
    return html

def calculate_and_save(username, age, gender, height_cm, weight_kg):
    if not username:
        return "Login first.", "", "", pd.DataFrame(columns=COLUMNS)

    if age is None or height_cm is None or weight_kg is None or not gender:
        return (
            "Please fill Age, Gender, Height and Weight.",
            "",
            "",
            pd.DataFrame(columns=COLUMNS),
        )

    df = load_data()
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    bmi = calculate_bmi(weight_kg, height_cm)
    category = classify_bmi(bmi)

    row = {
        "username": username,
        "age": age,
        "gender": gender,
        "height_cm": height_cm,
        "weight_kg": weight_kg,
        "bmi": bmi,
        "category": category,
        "datetime": now
    }

    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    save_data(df)

    result_html = make_result_card(username, bmi, category)
    advice = advice_for_category(category)

    return (
        "Saved successfully.",
        result_html,
        advice,
        pd.DataFrame([row])
    )

# --------------------------
# HISTORY + TREND PLOT
# --------------------------
def get_user_history(username):
    if not username:
        return "Login first.", pd.DataFrame(columns=COLUMNS), None

    df = load_data()
    user_df = df[df["username"] == username]

    if user_df.empty:
        return "No history found.", pd.DataFrame(columns=COLUMNS), None

    user_df = user_df.copy()
    user_df["datetime_parsed"] = pd.to_datetime(user_df["datetime"])
    user_df = user_df.sort_values("datetime_parsed")

    fig = plt.figure()
    colors = {
        "Underweight": "tab:blue",
        "Normal weight": "tab:green",
        "Overweight": "tab:orange",
        "Obese": "tab:red"
    }

    for cat in user_df["category"].unique():
        part = user_df[user_df["category"] == cat]
        plt.plot(part["datetime_parsed"], part["bmi"], marker="o",
                 label=cat, color=colors.get(cat, None))

    plt.title(f"BMI Trend for {username}")
    plt.xticks(rotation=45)
    plt.ylabel("BMI")
    plt.xlabel("Date")
    plt.legend(title="BMI Category")
    plt.tight_layout()

    return f"Loaded {username}'s history.", user_df[COLUMNS], fig

# --------------------------
# CLEAR HISTORY FOR USER
# --------------------------
def clear_my_history(username):
    if not username:
        return "Login first.", pd.DataFrame(columns=COLUMNS)

    df = load_data()
    df = df[df["username"] != username]
    save_data(df)

    return f"{username}'s history cleared.", df

# --------------------------
# CLEAR ALL DATA
# --------------------------
def clear_all_data():
    save_data(pd.DataFrame(columns=COLUMNS))
    return "All BMI data cleared.", pd.DataFrame(columns=COLUMNS)

# --------------------------
# EXPORT TO EXCEL
# --------------------------
def get_all_data():
    df = load_data()
    return f"Total {len(df)} records.", df

def export_excel():
    df = load_data()
    file = "bmi_export.xlsx"
    df.to_excel(file, index=False)
    return file

# ============================
#        UI STARTS HERE
# ============================

with gr.Blocks(title="BMI System") as demo:

    gr.Markdown("# 🧮 BMI Calculator with Login, Trends, Export & Clear Buttons")

    # small CSS styling for result card + form
    gr.HTML(
        """
        <style>

.bmi-card {
    max-width: 360px;            /* LIMIT CARD WIDTH */
    margin: 10px auto;           /* CENTER LEFT + RIGHT */
    background: linear-gradient(135deg, #eef2ff, #fdf2ff);
    border-radius: 18px;
    padding: 16px 18px;
    box-shadow: 0 10px 24px rgba(15, 23, 42, 0.10);
    font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
}

.bmi-header {
    display: flex;
    flex-direction: column;
    gap: 2px;
    margin-bottom: 8px;
}

.bmi-hello {
    font-size: 18px;
    font-weight: 700;
}

.bmi-small-text {
    font-size: 12px;
    opacity: 0.8;
}

.bmi-main {
    display: flex;
    align-items: baseline;
    justify-content: space-between;
    margin-top: 4px;
}

.bmi-number {
    font-size: 34px;
    font-weight: 800;
    letter-spacing: 0.02em;
}

.bmi-label-row {
    text-align: right;
}

.bmi-label {
    font-size: 12px;
    opacity: 0.8;
    display: block;
    margin-bottom: 4px;
}

.bmi-chip {
    display: inline-block;
    padding: 4px 10px;
    border-radius: 999px;
    font-size: 11px;
    font-weight: 600;
}

.bmi-chip-normal { background: #dcfce7; color: #166534; }
.bmi-chip-under { background: #dbeafe; color: #1d4ed8; }
.bmi-chip-over { background: #fef9c3; color: #854d0e; }
.bmi-chip-obese { background: #fee2e2; color: #b91c1c; }

.bmi-footer-text {
    margin-top: 10px;
    font-size: 12px;
    opacity: 0.9;
}

/* FORM CARD ALSO CENTERED + NARROWER */
.bmi-form-card {
    max-width: 420px;
    margin: 10px auto;           /* CENTER */
    background: #f9fafb;
    border-radius: 14px;
    padding: 12px 14px;
    border: 1px solid #e5e7eb;
}

.bmi-form-title {
    font-size: 14px;
    font-weight: 600;
    margin-bottom: 6px;
}

</style>

        """
    )

    current_user = gr.State("")

    with gr.Tabs() as tabs:
        # LOGIN TAB
        with gr.Tab("Login") as login_tab:
            u = gr.Textbox(label="Username", placeholder="Enter username")
            p = gr.Textbox(label="Password", type="password", placeholder="Enter password")
            btn = gr.Button("Login / Register", variant="primary")
            status = gr.Textbox(label="Status", interactive=False)
            display_user = gr.Textbox(label="Logged-in User", interactive=False)

        # BMI TAB (hidden until login)
        with gr.Tab("BMI Calculator", visible=False) as bmi_tab:
            gr.HTML(
                """
                <div class="bmi-form-card">
                    <div class="bmi-form-title">Fill your details</div>
                    <div style="font-size:12px; opacity:0.85;">
                        Enter your basics below and we will calculate your BMI instantly.
                    </div>
                </div>
                """
            )
            show_user = gr.Textbox(label="User", interactive=False)

            with gr.Row():
                age = gr.Number(label="Age", placeholder="Eg: 24")
                gender = gr.Radio(["Male", "Female", "Other"], label="Gender")

            with gr.Row():
                height = gr.Number(label="Height (cm)", placeholder="Eg: 170")
                weight = gr.Number(label="Weight (kg)", placeholder="Eg: 65")

            calc_btn = gr.Button("✨ Calculate & Save", variant="primary")

            out1 = gr.Textbox(label="Status", interactive=False)
            out2 = gr.HTML(label="Result")
            out3 = gr.Textbox(label="Advice", interactive=False)
            out4 = gr.Dataframe(label="Latest record")

        # HISTORY TAB (hidden until login)
        with gr.Tab("My History", visible=False) as hist_tab:
            hist_status = gr.Textbox(label="Status", interactive=False)
            hist_table = gr.Dataframe(label="My Data")
            hist_plot = gr.Plot(label="My Trend")
            load_hist_btn = gr.Button("Load My History")
            clear_my_btn = gr.Button("🗑 Clear My History", variant="stop")

        # ALL DATA TAB (hidden until login)
        with gr.Tab("All Data & Export", visible=False) as all_tab:
            all_status = gr.Textbox(label="Status")
            all_table = gr.Dataframe(label="All Records")
            refresh_btn = gr.Button("Refresh Data")

            export_btn = gr.Button("Export to Excel")
            excel_file = gr.File()

            clear_all_btn = gr.Button("🗑 Clear ALL Data", variant="stop")

    # --- Callbacks ---

    demo.load(lambda x: x, current_user, show_user)

    btn.click(
        handle_login,
        [u, p],
        [status, current_user, display_user, bmi_tab, hist_tab, all_tab, show_user]
    )

    calc_btn.click(
        calculate_and_save,
        [current_user, age, gender, height, weight],
        [out1, out2, out3, out4]
    )

    load_hist_btn.click(
        get_user_history, [current_user],
        [hist_status, hist_table, hist_plot]
    )

    clear_my_btn.click(
        clear_my_history, [current_user],
        [hist_status, hist_table]
    )

    refresh_btn.click(get_all_data, [], [all_status, all_table])
    export_btn.click(export_excel, [], excel_file)

    clear_all_btn.click(
        clear_all_data, [], [all_status, all_table]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://56623d4980585e7baa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
